In [20]:
import numpy as np
import pandas as pd
from transformers import DebertaV2Tokenizer, DebertaV2Model
import torch

In [2]:
tokenizer = DebertaV2Tokenizer.from_pretrained('./models/deberta-v2-xlarge')
model = DebertaV2Model.from_pretrained('./models/deberta-v2-xlarge')

Some weights of the model checkpoint at ./models/deberta-v2-xlarge were not used when initializing DebertaV2Model: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
ls local_data/unlabeled_contracts/2020

000000000.txt  000006365.txt  000012730.txt  000019095.txt  000025460.txt
000000001.txt  000006366.txt  000012731.txt  000019096.txt  000025461.txt
000000002.txt  000006367.txt  000012732.txt  000019097.txt  000025462.txt
000000003.txt  000006368.txt  000012733.txt  000019098.txt  000025463.txt
000000004.txt  000006369.txt  000012734.txt  000019099.txt  000025464.txt
000000005.txt  000006370.txt  000012735.txt  000019100.txt  000025465.txt
000000006.txt  000006371.txt  000012736.txt  000019101.txt  000025466.txt
000000007.txt  000006372.txt  000012737.txt  000019102.txt  000025467.txt
000000008.txt  000006373.txt  000012738.txt  000019103.txt  000025468.txt
000000009.txt  000006374.txt  000012739.txt  000019104.txt  000025469.txt
000000010.txt  000006375.txt  000012740.txt  000019105.txt  000025470.txt
000000011.txt  000006376.txt  000012741.txt  000019106.txt  000025471.txt
000000012.txt  000006377.txt  000012742.txt  000019107.txt  000025472.txt
000000013.txt  000006378.txt  00001274

In [8]:
filepath = 'local_data/unlabeled_contracts/2020/000019119.txt'

In [14]:
with open(filepath, 'r', encoding="utf-8") as file:
    contract = file.read().replace('\n', '')

In [15]:
contract

'Exhibit 10.22\xa0CERTAIN CONFIDENTIAL INFORMATION CONTAINED IN THIS DOCUMENT, MARKED BY [***],HAS BEEN OMITTED BECAUSE IT IS BOTH (I) NOT MATERIAL AND (II)\xa0WOULD LIKELY CAUSECOMPETITIVE HARM IF PUBLICLY DISCLOSED.\xa0AMENDMENT NO. 1 TO THE COMMON STOCK ISSUANCE AGREEMENTThis Amendment No. 1 to the Common Stock Issuance Agreement (“Amendment”) ismade and entered into, effective as of December 17, 2019 (“Amendment EffectiveDate”), by and between Vir Biotechnology, Inc., a Delaware corporation withoffices at with an office at 499 Illinois Street, San Francisco, California94158 (“Vir”), and Alnylam Pharmaceuticals, Inc., a Delaware corporation locatedat 300 Third Street, Cambridge, Massachusetts 02142 (“Alnylam”). Each of Vir andAlnylam are referred to in this Amendment as a “Party” and together, the“Parties”.BackgroundWHEREAS, the Parties have entered into that certain Collaboration and LicenseAgreement effective as of October 16, 2017 (as amended by Letter Agreement datedNovember 13,

In [18]:
def get_questions_from_csv():
    df = pd.read_csv("./data/category_descriptions.csv")
    q_dict = {}
    for i in range(df.shape[0]):
        category = df.iloc[i, 0].split("Category: ")[1]
        description = df.iloc[i, 1].split("Description: ")[1]
        q_dict[category.title()] = description
    return q_dict

In [25]:
qtype_dict = get_questions_from_csv()
qtype_dict['Document Name']

'The name of the contract'

In [16]:
inputs = tokenizer(contract, return_tensors='pt')
outputs = model(**inputs)

Token indices sequence length is longer than the specified maximum sequence length for this model (1017 > 512). Running this sequence through the model will result in indexing errors


In [17]:
outputs

BaseModelOutput(last_hidden_state=tensor([[[ 0.2511, -0.4658,  0.3150,  ..., -0.2153,  1.0133,  2.1668],
         [ 0.2539, -0.8238,  0.2599,  ..., -0.0088,  1.0295,  2.1480],
         [ 0.3727, -0.6130,  0.1983,  ...,  0.1297,  0.9709,  2.0877],
         ...,
         [ 0.1094, -0.3109,  0.4702,  ..., -0.0915,  1.1152,  1.8452],
         [ 0.0711, -0.1317, -0.0174,  ..., -0.3326,  1.4733,  1.7046],
         [-0.0047, -0.2950, -0.3134,  ..., -0.8848,  0.8114,  1.3031]]],
       grad_fn=<NativeLayerNormBackward>), hidden_states=None, attentions=None)